In [ ]:
# Send a message to Confluent-Kafka Topic via Python

In [ ]:
# Required connection configs for Kafka producer, consumer, and admin
bootstrap.servers=pkc-921jm.us-east-2.aws.confluent.cloud:9092
security.protocol=SASL_SSL
sasl.mechanisms=PLAIN
sasl.username=FLY6JUGGUCNYPEC4
sasl.password=ag/kH0DDkMvDUIt3zscBtcDhct9ZrzzQjzFIzNIMcgGIkbKyc7ZTPQ/dF6tOGnoO

# Best practice for higher availability in librdkafka clients prior to 1.7
session.timeout.ms=45000

client.id=ccloud-python-client-0ba89b25-4c4f-437d-b6a2-47efab204258


In [1]:
!pip install confluent-Kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 21.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import json

csv_file = 'Customers_data/first_100_customers.csv'

df = pd.read_csv(csv_file)

df.head()








,customer_id,name,city,state,country,registration_date,is_active
0,0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False


In [4]:
json_records = df.to_dict(orient='records')

json_file = 'customers.json'

with open(json_file,'w') as file:
  json.dump(json_records,file,indent=4)


print("File Converted to JSON")

File Converted to JSON


In [ ]:
# prompt: give a code to provide kafka producer conf

import pandas as pd
import json
from confluent_kafka import Producer

# Kafka Configuration
kafka_config = {
    'bootstrap.servers': 'pkc-921jm.us-east-2.aws.confluent.cloud:9092',
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'FLY6JUGGUCNYPEC4',
    'sasl.password': 'ag/kH0DDkMvDUIt3zscBtcDhct9ZrzzQjzFIzNIMcgGIkbKyc7ZTPQ/dF6tOGnoO',
    'session.timeout.ms': 45000,
    'client.id': 'ccloud-python-client-0ba89b25-4c4f-437d-b6a2-47efab204258'
}

# ... (rest of your code)


In [7]:
from confluent_kafka import Producer
import json
import time



conf ={
  "bootstrap.servers":"pkc-921jm.us-east-2.aws.confluent.cloud:9092",
  "security.protocol":"SASL_SSL",
  "sasl.mechanisms":"PLAIN",
  "sasl.username":"FLY6JUGGUCNYPEC4",
  "sasl.password":"ag/kH0DDkMvDUIt3zscBtcDhct9ZrzzQjzFIzNIMcgGIkbKyc7ZTPQ/dF6tOGnoO",
  'session.timeout.ms': 45000,
  'client.id': 'ccloud-python-client-0ba89b25-4c4f-437d-b6a2-47efab204258'
}

producer = Producer(conf)

In [9]:
topic = 'ecommerce'

with open('customers.json','r') as file:
  customers_data = json.load(file)

value = customers_data[0]
key = value['customer_id']

print(key,value)

0 {'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}


In [13]:
type(str(value).encode('utf-8'))

bytes

In [14]:
producer.produce(topic,key = str(key).encode('utf-8'),value=str(value).encode('utf-8'))

In [ ]:
# Send multiple messages to our Kafka Cluster


In [18]:
customers_data

[{'customer_id': 0,
  'name': 'Customer_0',
  'city': 'Pune',
  'state': 'Maharashtra',
  'country': 'India',
  'registration_date': '2023-06-29',
  'is_active': False},
 {'customer_id': 1,
  'name': 'Customer_1',
  'city': 'Bangalore',
  'state': 'Tamil Nadu',
  'country': 'India',
  'registration_date': '2023-12-07',
  'is_active': True},
 {'customer_id': 2,
  'name': 'Customer_2',
  'city': 'Hyderabad',
  'state': 'Gujarat',
  'country': 'India',
  'registration_date': '2023-10-27',
  'is_active': True},
 {'customer_id': 3,
  'name': 'Customer_3',
  'city': 'Bangalore',
  'state': 'Karnataka',
  'country': 'India',
  'registration_date': '2023-10-17',
  'is_active': False},
 {'customer_id': 4,
  'name': 'Customer_4',
  'city': 'Ahmedabad',
  'state': 'Karnataka',
  'country': 'India',
  'registration_date': '2023-03-14',
  'is_active': False},
 {'customer_id': 5,
  'name': 'Customer_5',
  'city': 'Hyderabad',
  'state': 'Karnataka',
  'country': 'India',
  'registration_date': '2023

In [20]:

topic = 'ecommerce'

# with open('customers.json','r') as file:
#   customers_data = json.load(file)

def delivery_status(err,msg):
  if(err):
    print(f"Message delivery failed: {err}")
  else:
    print(f"message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

for record in customers_data:
  try:

    message_value = json.dumps(record)
    message_key = str(int(record['customer_id']) + 1000).encode('utf-8')

    producer.produce(topic,key = message_key,value=message_value,callback = delivery_status)
    producer.poll(1)

  except Exception as e:
    print(f"Error sending messages: {e}")

producer.flush()

print("Message send to kafka successfully")

message delivered to ecommerce [0] at offset 42
message delivered to ecommerce [0] at offset 43
message delivered to ecommerce [2] at offset 26
message delivered to ecommerce [2] at offset 27
message delivered to ecommerce [1] at offset 38
message delivered to ecommerce [2] at offset 28
message delivered to ecommerce [0] at offset 44
message delivered to ecommerce [1] at offset 39
message delivered to ecommerce [2] at offset 29
message delivered to ecommerce [2] at offset 30
message delivered to ecommerce [0] at offset 45
message delivered to ecommerce [1] at offset 40
message delivered to ecommerce [2] at offset 31
message delivered to ecommerce [2] at offset 32
message delivered to ecommerce [2] at offset 33
message delivered to ecommerce [2] at offset 34
message delivered to ecommerce [2] at offset 35
message delivered to ecommerce [1] at offset 41
message delivered to ecommerce [1] at offset 42
message delivered to ecommerce [0] at offset 46
message delivered to ecommerce [1] at of